# Let´s do some explorations on different optimization use cases

Ressources: https://towardsdatascience.com/how-to-develop-optimization-models-in-python-1a03ef72f5b4

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import pulp
from pulp import *

## Use Case 1:
- A bakery makes cakes and pies every day. 
- It can make a total of 30 items in one day, which at least must be 5 cakes and 10 pies for its planned customers. 
- The profit on each cake is $1.5 and the profit on each pie is $2.00. 
- How many cakes and pies should be made to maximize the profit?

### Solution Approach #1: Monte-Carlo Simulation using Random Search

In [2]:
cakes = []
pies = []
profits = []
n = 100
# loop over random number of cakes between 0<= c <=30
for _ in range(n):
    c = int(np.random.uniform(low=0, high=30, size=1)) 
    # loop over random number of pies between 0<= p <=30
    for _ in range(n):
        p = int(np.random.uniform(low=0, high=30, size=1))
        # check boundary conditions
        if c + p <= 30 and c>=5 and p>=10:
            profit = 1.5*c + 2.0*p
            cakes.append(c)
            pies.append(p)
            profits.append(profit)

# make dataframe
d = {'cakes': cakes, 'pies': pies, 'profit': profits}
mcs_results = pd.DataFrame(data=d)
# drop duplicates
mcs_results = mcs_results.drop_duplicates()
# sort by profits
mcs_results = mcs_results.sort_values(by='profit', ascending=False) 
# find best parameters to optimize profit
p_opt = int(mcs_results.iloc[0,:].pies)
c_opt  = int(mcs_results.iloc[0,:].cakes)
profit_max = mcs_results.iloc[0,:].profit
print('The bakery should bake {} pies and {} cakes to maximize profit of {} $'.format(p_opt, c_opt, profit_max))         

The bakery should bake 25 pies and 5 cakes to maximize profit of 57.5 $


In [3]:
# find maximum profit
mcs_results.head(5)

,cakes,pies,profit
462,5,25,57.5
209,6,24,57.0
166,7,23,56.5
294,8,22,56.0
437,5,24,55.5


In [4]:
fig = px.histogram(mcs_results, x='profit')
fig.update_layout(width=500,height=500)
fig.show()

In [5]:
mcs_results_mat = mcs_results.pivot(index='cakes', columns='pies')['profit'].fillna(0)
mcs_results_mat

pies,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
cakes,,,,,,,,,,,,,,,,
5,27.5,29.5,31.5,33.5,35.5,37.5,39.5,41.5,43.5,45.5,47.5,49.5,51.5,53.5,55.5,57.5
6,29.0,31.0,33.0,35.0,37.0,39.0,41.0,43.0,45.0,47.0,49.0,51.0,53.0,55.0,57.0,0.0
7,30.5,32.5,34.5,36.5,38.5,40.5,42.5,44.5,46.5,48.5,50.5,52.5,54.5,56.5,0.0,0.0
8,32.0,34.0,36.0,38.0,40.0,42.0,44.0,46.0,48.0,50.0,52.0,54.0,56.0,0.0,0.0,0.0
9,33.5,35.5,37.5,39.5,41.5,43.5,45.5,47.5,49.5,51.5,53.5,55.5,0.0,0.0,0.0,0.0
10,35.0,37.0,39.0,41.0,0.0,45.0,47.0,49.0,51.0,53.0,55.0,0.0,0.0,0.0,0.0,0.0
11,36.5,38.5,40.5,42.5,44.5,46.5,48.5,50.5,52.5,54.5,0.0,0.0,0.0,0.0,0.0,0.0
12,38.0,40.0,42.0,44.0,46.0,48.0,50.0,52.0,54.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,39.5,41.5,43.5,45.5,47.5,49.5,51.5,53.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
fig = px.imshow(mcs_results_mat, x=mcs_results_mat.columns, y=mcs_results_mat.index)
fig.update_layout(width=500,height=500)
fig.show()

### Solution Approach #2: Linear Optimization using PupLP

In [7]:
model = LpProblem('Maximize Bakery Profits', sense= LpMaximize)

c:\Users\Sebas\anaconda3\envs\optimization_env\lib\site-packages\pulp\pulp.py:1352: UserWarning:

Spaces are not permitted in the name. Converted to '_'



In [8]:
# define decision variables
c = LpVariable('C', lowBound=0, upBound=None, cat='Integer')
p = LpVariable('P', lowBound=0, upBound=None, cat='Integer')

In [9]:
# define constraints
model += c + p <= 30 # at least 30 items
model += c >= 5 # at least 5 cakes
model += p >= 10 # at least 10 pies

In [10]:
# define objective function
model += 1.5 * c + 2.0 * p

In [11]:
# solve model
print('solve model...')
print('model status: ', LpStatus[model.status])
model.solve()
print('model status: ', LpStatus[model.status])
print('optimized number of cakes: ', int(c.varValue))
print('optimized number of pies: ', int(p.varValue))
print('maximized price: ', value(model.objective))

solve model...
model status:  Not Solved
model status:  Optimal
optimized number of cakes:  5
optimized number of pies:  25
maximized price:  57.5


## Use Case 2:
- A post office is looking to hire postman, with the requirements to work 5 consecutive days and then 2 days off. 
- The estimated least number of postmen needed for each day of the week is: Monday: 25, Tuesday: 32, Wednesday: 22, Thursday: 18, Friday: 24, Saturday: 12, Sunday: 14
- The objective is to hire the minimum number of workers

To solve this problem, we need to write down the constraints in terms of the number of workers we need to start working on each day such as; x_0 is the number of workers starting to work on Monday, x_1 is the number of workers starting to work on Tuesday, etc. By doing so, we can store x_0 from Monday to Friday and x_1 from Tuesday to Saturday since they need to work 5 consecutive days

### Solution Approach #1: Monte-Carlo Simulation using Random Search

In [25]:
# let´s treat the number of workers for each week day as independent variables

workers_mon = []
workers_tue = []
workers_wed = []
workers_thu = []
workers_fri = []
workers_sat = []
workers_sun = []
workers = []
n = 10
# loop over random number of monday workers between 0 and 25 workers
for _ in range(n):
    wmon = int(np.random.uniform(low=0, high=25, size=1)) 
    # # loop over random number of tuesday workers between 0 and 32 workers
    for _ in range(n):
        wtue = int(np.random.uniform(low=0, high=32, size=1))
        # loop over random number of wednesday workers between 0 and 22 workers
        for _ in range(n):
            wwed = int(np.random.uniform(low=0, high=22, size=1))
            # loop over random number of thursday workers between 0 and 18 workers
            for _ in range(n):
                wthu = int(np.random.uniform(low=0, high=18, size=1))
                # loop over random number of friday workers between 0 and 24 workers
                for _ in range(n):
                    wfri = int(np.random.uniform(low=0, high=24, size=1))
                    # loop over random number of saturday workers between 0 and 12 workers
                    for _ in range(n):
                        wsat = int(np.random.uniform(low=0, high=12, size=1))
                        # loop over random number of sunday workers between 0 and 12 workers
                        for _ in range(n):
                            wsun = int(np.random.uniform(low=0, high=14, size=1))
        
                            # check boundary conditions          
                            cond0 = wmon + wthu + wfri + wsat + wsun >= 25
                            cond1 = wmon + wtue + wfri + wsat + wsun >= 32
                            cond2 = wmon + wthu + wwed + wsat + wsun >= 22
                            cond3 = wmon + wthu + wwed + wthu + wsun >= 18
                            cond4 = wmon + wthu + wwed + wthu + wfri >= 24
                            cond5 = wtue + wwed + wthu + wfri + wsat >= 12
                            cond6 = wwed + wthu + wfri + wsat + wsun >= 14
                            
                            if cond0 and cond1 and cond2 and cond3 and cond4 and cond5 and cond6:
                                # total number of workers
                                num_workers = wmon+wtue+wwed+wthu+wfri+wsat+wsun
                                # append number of workers for each week day
                                workers_mon.append(wmon)
                                workers_tue.append(wtue)
                                workers_wed.append(wwed)
                                workers_thu.append(wthu)
                                workers_fri.append(wfri)
                                workers_sat.append(wsat)
                                workers_sun.append(wsun)
                                workers.append(num_workers)




In [ ]:
# make dataframe
d = {'cakes': cakes, 'pies': pies, 'profit': profits}
mcs_results = pd.DataFrame(data=d)
# drop duplicates
mcs_results = mcs_results.drop_duplicates()
# sort by profits
mcs_results = mcs_results.sort_values(by='profit', ascending=False) 
# find best parameters to optimize profit
p_opt = int(mcs_results.iloc[0,:].pies)
c_opt  = int(mcs_results.iloc[0,:].cakes)
profit_max = mcs_results.iloc[0,:].profit
print('The bakery should bake {} pies and {} cakes to maximize profit of {} $'.format(p_opt, c_opt, profit_max))

### Solution Approach #2: Linear Optimization using PupLP

In [12]:
#Initialize model
model = LpProblem("Minimize Number of Workers", LpMinimize)
#Define decision variables
days = list(range(7))

In [13]:
x = LpVariable.dicts('workers_', days, lowBound=0, upBound=None, cat='Integer')

In [14]:
x

{0: workers__0,
 1: workers__1,
 2: workers__2,
 3: workers__3,
 4: workers__4,
 5: workers__5,
 6: workers__6}

In [15]:
[x[i] for i in days]

[workers__0,
 workers__1,
 workers__2,
 workers__3,
 workers__4,
 workers__5,
 workers__6]

In [16]:
# Define objective function/model
model += lpSum([x[i] for i in days])

In [17]:
# Define constraints
model += x[0] + x[3] + x[4] + x[5] + x[6] >= 25
model += x[0] + x[1] + x[4] + x[5] + x[6] >= 32
model += x[0] + x[1] + x[2] + x[5] + x[6] >= 22
model += x[0] + x[1] + x[2] + x[3] + x[6] >= 18
model += x[0] + x[1] + x[2] + x[3] + x[4] >= 24
model += x[1] + x[2] + x[3] + x[4] + x[5] >= 12
model += x[2] + x[3] + x[4] + x[5] + x[6] >= 14

In [18]:
# Solve model
model.solve()
#Print model status
print('Status:', LpStatus[model.status])

Status: Optimal


In [19]:
# Print solution variables
for variable in model.variables():
    print ('{} = {}'.format(variable.name, variable.varValue))

workers__0 = 7.0
workers__1 = 7.0
workers__2 = 0.0
workers__3 = 0.0
workers__4 = 10.0
workers__5 = 0.0
workers__6 = 8.0


In [20]:
# The least total number of workers required is: 
workers = 0
for variable in model.variables():
    workers += variable.varValue
print('The least total number of workers required is: ', int(workers))

The least total number of workers required is:  32
